# Loading election results from a spreadsheet

In [1]:
import pyexcel as p
records=p.iget_records(file_name="2017 General Election Results.xls")

In [2]:
def filter_2017(records):
    for record in records:
        if record['Year'] == 2017 and record['Majority Party'] != '':
            if record['Constituency'].startswith('Ynys'):
                record['Constituency'] = 'Ynys Mon'
            yield record
        
sheet=p.iget_array(records=filter_2017(records))
sheet=list(sheet)

In [3]:
len(sheet)

651

In [4]:
s=p.Sheet(sheet)

In [5]:
def filter_row(row_index, row):
    return row[1] == ''

del s.row[filter_row]

In [6]:
s.column.select([2, 9])
s.top(20)

Constituency,Majority Party
Aldershot,C
Aldridge-Brownhills,C
Altrincham and Sale West,C
Amber Valley,C
Arundel and South Downs,C
Ashfield,Lab
Ashford,C
Ashton-under-Lyne,Lab
Aylesbury,C
Banbury,C


In [7]:
# remove header
del s.row[0]

In [8]:
set(s.column[1])

{'C', 'DUP', 'Green', 'Ind', 'LD', 'Lab', 'PC', 'SF', 'SNP'}

In [9]:
from pyecharts_extras import ChoroplethMap

# piece wise configuration
pieces = [
  {"tag": "C", 'label': "Conservative", "color": '#0E5EBD'},
  {"tag":  "PC", 'label': "Plaid Cymru", "color": '#45a86d' },
   { "tag":  "LD", 'label': "Liberal Democrat", 'color': '#dc5006'},
   { "tag": "Lab", 'label': "Labour", "color": '#e5000e'},
   { "tag":"DUP", "label": "Democratic Unionist Party", "color": '#b0002f'},
   { "tag": "Green", "label": "Green", "color": '#136326'},
    {"tag":"SNP", "label": "Scottish National Party", "color": '#f4d614'},
    {"tag":"Ind", "label": "Independent Party : Speaker", "color": 'gray'},
    {"tag":"SF", "label": "Sinn Fein", "color": '#094428'}
]

In [10]:
from pyecharts import options as opts
from pyecharts.datasets import register_url
from echarts_united_kingdom_pypkg import NM_WESTMINSTER_2016_UK

register_url('https://echarts-maps.github.io/echarts-united-kingdom-js')

map = (
    ChoroplethMap(
        init_opts=opts.InitOpts(
           height="700px", width="600px", bg_color="#c5d6e7"
        )
      )
     .set_global_opts(title_opts=opts.TitleOpts(
         title='Results of the 2017 General Election', 
         subtitle='The governing Conservative Party narrowly won the election, remaining the single largest party, \nbut lost its majority resulting in the formation of a minority government with the support of \nthe Democratic Unionist Party (DUP) of Northern Ireland.',
         ))
     .add('', zip(s.column[0], s.column[1]), maptype='UK_electoral_2016', 
        choropleth_legend=pieces,
        is_map_symbol_show=False,
        tooltip_opts=opts.TooltipOpts(
            textstyle_opts=opts.TextStyleOpts(color='#000000'),
            background_color='white',
            border_color='black',
            border_width=1,
            formatter='{b}'
        ),
        emphasis_itemstyle_opts=opts.ItemStyleOpts(border_color="black"),
        label_opts=opts.LabelOpts(is_show=False),
        name_map=NM_WESTMINSTER_2016_UK,
        itemstyle_opts=opts.ItemStyleOpts(border_color="white"),
        emphasis_label_opts=opts.LabelOpts(is_show=False)
    )
)

map.render_notebook()